# Load the dataset

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

# Download the flower dataset
data_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
path_to_zip = keras.utils.get_file('flower_photos.tgz', origin=data_url, extract=True)
base_dir = path_to_zip.replace('flower_photos.tgz', 'flower_photos')

# Create training and validation datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    base_dir,
    image_size=(128, 128),
    batch_size=32,
    seed=123,
    validation_split=0.2,
    subset='training'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    base_dir,
    image_size=(128, 128),
    batch_size=32,
    seed=123,
    validation_split=0.2,
    subset='validation'
)


Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


# Preprocessing

In [2]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Apply normalization
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


# Build the Neural Network Model

In [3]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(5, activation='softmax')  # 5 classes for the flowers
])

# Compile the model

In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model

In [5]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
92/92 [==============================] - 20s 202ms/step - loss: 1.2807 - accuracy: 0.4506 - val_loss: 1.0569 - val_accuracy: 0.5695
Epoch 2/10
92/92 [==============================] - 20s 218ms/step - loss: 0.9792 - accuracy: 0.6114 - val_loss: 0.9747 - val_accuracy: 0.6294
Epoch 3/10
92/92 [==============================] - 20s 219ms/step - loss: 0.8291 - accuracy: 0.6798 - val_loss: 0.8864 - val_accuracy: 0.6594
Epoch 4/10
92/92 [==============================] - 20s 218ms/step - loss: 0.7107 - accuracy: 0.7279 - val_loss: 0.8693 - val_accuracy: 0.6608
Epoch 5/10
92/92 [==============================] - 20s 220ms/step - loss: 0.5287 - accuracy: 0.8048 - val_loss: 0.8937 - val_accuracy: 0.6662
Epoch 6/10
92/92 [==============================] - 20s 220ms/step - loss: 0.3582 - accuracy: 0.8750 - val_loss: 0.9837 - val_accuracy: 0.6730
Epoch 7/10
92/92 [==============================] - 20s 217ms/step - loss: 0.2466 - accuracy: 0.9121 - val_loss: 1.3920 - val_accuracy: 0.6117

# Evaluate the model

In [6]:
test_loss, test_accuracy = model.evaluate(val_ds)
print(f"Validation accuracy: {test_accuracy:.2f}")

23/23 [==============================] - 1s 48ms/step - loss: 1.5834 - accuracy: 0.7016
Validation accuracy: 0.70


# Visualize Training Results

In [ ]:
def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.show()

plot_training_history(history)

# Classify an external image

In [ ]:
# Function to load and preprocess an external image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match model input
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to classify an external image
def classify_image(model, img_path):
    processed_image = load_and_preprocess_image(img_path)
    predictions = model.predict(processed_image)  # Get predictions
    predicted_class = np.argmax(predictions[0])  # Get the index of the highest predicted value
    return predicted_class

# Example usage
img_path = 'path_to_your_image.jpg'  # Replace with your image path
predicted_class = classify_image(model, img_path)
print(f"The predicted class is: {predicted_class}")